Radiology classification

In [2]:
# Import Libraries
import string
import io
import pandas as pd

/Users/h6x/anaconda3/envs/TDA/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# Load the data
notefiles_df = pd.read_csv('/Users/h6x/UT/Coursework/2025/Spring 2025/COSC 525 - Data Mining/git/cosc-526-data-engineering-project/data/physionet.org/files/mimiciii/1.4/NOTEEVENTS.csv')

/var/folders/j3/904q7dbj02j_jyjz7pvfgz384d40kw/T/ipykernel_24097/3906576199.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  notefiles_df = pd.read_csv('/Users/h6x/UT/Coursework/2025/Spring 2025/COSC 525 - Data Mining/git/cosc-526-data-engineering-project/data/physionet.org/files/mimiciii/1.4/NOTEEVENTS.csv')


In [4]:
# mask for Radiology reports
mask = notefiles_df['CATEGORY'] == 'Radiology'
radiology_df = notefiles_df[mask]

In [5]:
radiology_df['DESCRIPTION'].value_counts().reset_index().head(10)

,DESCRIPTION,count
0,CHEST (PORTABLE AP),169270
1,CHEST (PA & LAT),43158
2,CT HEAD W/O CONTRAST,34485
3,CHEST PORT. LINE PLACEMENT,21596
4,PORTABLE ABDOMEN,8143
5,CHEST (PRE-OP PA & LAT),8064
6,CT CHEST W/CONTRAST,8001
7,CT ABDOMEN W/CONTRAST,7304
8,MR HEAD W & W/O CONTRAST,7062
9,CT CHEST W/O CONTRAST,6745


Filter into few classes

In [ ]:
description_filter = ['CHEST (PA & LAT)','CT HEAD W/O CONTRAST','CHEST PORT. LINE PLACEMENT']

filtered_radiology_df = radiology_df[radiology_df['DESCRIPTION'].isin(description_filter)]

In [16]:
# get 2000 from each type of report
filtered_radiology_df = filtered_radiology_df.groupby('DESCRIPTION').head(2000)

In [17]:
filtered_radiology_df.shape

(6000, 11)

Load pre trained Bert models

In [1]:
from transformers import BertTokenizer, BertModel
import torch

In [8]:
from transformers import AutoTokenizer, AutoModel

In [9]:
# Load ClinicalBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model.eval()

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

Generate embeddings for entire DataFrame


In [10]:
import numpy as np

In [11]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use [CLS] token embedding (outputs[0][:,0,:]) or mean pooling
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # Shape: (1, 768)
    return cls_embedding.squeeze().numpy()

In [12]:
filtered_radiology_df.head(3)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
728605,738663,20409,102314.0,2119-03-11,2119-03-11 11:46:00,NaN,Radiology,CT HEAD W/O CONTRAST,NaN,NaN,[**2119-3-11**] 11:46 AM\n CT HEAD W/O CONTRAS...
732767,739780,24248,NaN,2158-05-23,2158-05-23 16:00:00,NaN,Radiology,CHEST (PA & LAT),NaN,NaN,[**2158-5-23**] 4:00 PM\n CHEST (PA & LAT) ...
732771,739820,7534,194035.0,2160-05-23,2160-05-23 11:07:00,NaN,Radiology,CHEST (PA & LAT),NaN,NaN,[**2160-5-23**] 11:07 AM\n CHEST (PA & LAT) ...


In [19]:
# # Create a list of embeddings
# embeddings = np.vstack([get_embedding(text) for text in filtered_radiology_df['TEXT']])

from tqdm import tqdm

# Add tqdm to your list comprehension
embeddings = np.vstack([get_embedding(text) for text in tqdm(filtered_radiology_df['TEXT'], desc="Embedding reports")])


Train your classifier

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [22]:
# Encode labels (chest -> 0, head -> 1, etc.)
le = LabelEncoder()
y = le.fit_transform(filtered_radiology_df['DESCRIPTION'])

In [23]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.2, random_state=42)

In [27]:
# Train classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [28]:
# Predict and evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))

                            precision    recall  f1-score   support

          CHEST (PA & LAT)       0.99      0.98      0.99       406
CHEST PORT. LINE PLACEMENT       0.98      0.99      0.99       383
      CT HEAD W/O CONTRAST       1.00      1.00      1.00       411

                  accuracy                           0.99      1200
                 macro avg       0.99      0.99      0.99      1200
              weighted avg       0.99      0.99      0.99      1200

